# Connect to historic data API

In [1]:
# Free API, no credentials needed
import yfinance as yf

import pandas as pd
import utils
from datapackage import Package, Resource

/Users/come.vitis/Documents/Personnal/Projects/finance-env/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Constants
TICKERS = {
    "S&P500": "^GSPC"
}

In [ ]:
# Load heavy resources here
# Get the constituents of S&P500 
sp500_constituents = utils.load_sp_500_constituents() # Default scrap mode is false

In [5]:
# Arbitrarly for now: select 5 companies
top_5 = sp500_constituents[:5]

   Unnamed: 0 symbol              company       sector
0           0    MMM                   3M  Industrials
1           1    AOS          A. O. Smith  Industrials
2           2    ABT  Abbott Laboratories  Health Care
3           3   ABBV               AbbVie  Health Care
4           4   ABMD              Abiomed  Health Care


In [4]:
# All information here : https://pypi.org/project/yfinance/